In [2]:
import keras
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers

from __future__ import print_function
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import RMSprop, SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from emnist import extract_training_samples
from emnist import extract_test_samples

import matplotlib.pyplot as plt
np.random.seed(1671)  # for reproducibility

ModuleNotFoundError: No module named 'keras'

In [1]:

# network and training
NB_EPOCH = 100
BATCH_SIZE = 256
VERBOSE = 2
NB_CLASSES = 47   # Number of classes in EMNIST dataset (letters and digits)
OPTIMIZER = Adam()  # optimizer, explained in this chapter
N_HIDDEN = 512
VALIDATION_SPLIT = 0.2  # how much TRAIN is reserved for VALIDATION
DROPOUT = 0.20

# Load EMNIST dataset
X_train, y_train = extract_training_samples('letters')
X_test, y_test = extract_test_samples('letters')

# Reshape and preprocess the data
RESHAPED = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], RESHAPED)
X_test = X_test.reshape(X_test.shape[0], RESHAPED)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Convert labels to start from 0
y_train = y_train - 1
y_test = y_test - 1

# Convert class vectors to binary class matrices
Y_train = to_categorical(y_train, NB_CLASSES)
Y_test = to_categorical(y_test, NB_CLASSES)

# Build the model
model = Sequential()
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,)))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE, epochs=NB_EPOCH,
                    verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# Evaluate the model on the test set
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

# Plot accuracy vs. epoch
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

# Plot loss vs. epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Save the trained model
model.save('trained_model.h5')
print("Model saved successfully.")
# Evaluación del modelo
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

NameError: name 'Adam' is not defined

In [ ]:
model.save('my_model.keras')
# Seleccionar una muestra aleatoria del conjunto de prueba
index = np.random.randint(0, len(X_test))
sample = X_test[index].reshape(1, -1)  # Reshape para que sea una fila

# Realizar la predicción usando el modelo cargado
prediction = model.predict(sample)

# Decodificar la predicción para obtener la clase predicha
predicted_class = np.argmax(prediction)

# Mostrar la letra y la predicción al usuario
plt.imshow(X_test[index].reshape(28, 28), cmap='gray')
plt.title(f'Predicción: {chr(predicted_class + 65)}')  # Convertir a letra (A-Z)
plt.show()


In [ ]:
# Seleccionar una muestra aleatoria del conjunto de prueba
index = np.random.randint(0, len(X_test))
sample = X_test[index].reshape(1, -1)  # Reshape para que sea una fila

# Realizar la predicción usando el modelo cargado
prediction = loaded_model.predict(sample)

# Decodificar la predicción para obtener la clase predicha
predicted_class = np.argmax(prediction)

# Mostrar la letra y la predicción al usuario
plt.imshow(X_test[index].reshape(28, 28), cmap='gray')
plt.title(f'Predicción: {chr(predicted_class + 65)}')  # Convertir a letra (A-Z)
plt.show()